In [91]:
import pandas as pd
from math import sqrt


In [92]:

# بارگذاری داده‌ها
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')


In [93]:

# پردازش داده‌های فیلم‌ها
movies_df['year'] = movies_df['title'].str.extract(r'(\(\d{4}\))', expand=False)
movies_df['year'] = movies_df['year'].str.extract(r'(\d{4})', expand=False)
movies_df['title'] = movies_df['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()


In [94]:

# حذف ستون 'genres' و 'timestamp'
movies_df = movies_df.drop('genres', axis=1)
ratings_df = ratings_df.drop('timestamp', axis=1)


In [95]:

# حذف مقادیر غیرعددی در ستون 'userId'
ratings_df['userId'] = pd.to_numeric(ratings_df['userId'], errors='coerce')  # مقادیر غیرعددی به NaN تبدیل می‌شوند
ratings_df = ratings_df.dropna(subset=['userId'])  # حذف ردیف‌هایی که userId آن‌ها NaN است
ratings_df['userId'] = ratings_df['userId'].astype(int)  # تبدیل به نوع int پس از حذف مقادیر غیرعددی


In [96]:

# ورودی نمونه از کاربر
userInput = [
    {'title': 'The Breakfast Club', 'rating': 5},
    {'title': 'Toy Story', 'rating': 3.5},
    {'title': 'Jumanji', 'rating': 2},
    {'title': 'Pulp Fiction', 'rating': 5},
    {'title': 'Akira', 'rating': 4.5}
]
inputMovies = pd.DataFrame(userInput)


In [97]:

# پیدا کردن MovieID برای فیلم‌های ورودی
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies, on='title')  # اضافه کردن on='title' برای ادغام درست


In [98]:

# حذف ستون 'year'
inputMovies = inputMovies.drop('year', axis=1)

# فیلتر کردن کاربران که این فیلم‌ها را تماشا کرده‌اند
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]

# گروه‌بندی کاربران بر اساس 'userId'
userSubsetGroup = userSubset.groupby(['userId'])

# محاسبه همبستگی پیرسون بین کاربر ورودی و سایر کاربران
pearsonCorrelationDict = {}


In [99]:

for name, group in userSubsetGroup:
    # مرتب‌سازی داده‌ها بر اساس 'movieId'
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    # پیدا کردن فیلم‌هایی که کاربر ورودی و کاربر مورد نظر هر دو دیده‌اند
    nRatings = len(group)
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    if len(temp_df) == 0:
        continue
    
    # به‌دست آوردن امتیازات هر دو کاربر
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()

    # محاسبه Sxx, Syy, Sxy
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList), 2)/float(nRatings)
    Sxy = sum(i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    # محاسبه همبستگی
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [100]:

# ساخت دیتافریم برای همبستگی کاربران
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index', columns=['similarityIndex'])
pearsonDF['userId'] = pearsonDF.index
pearsonDF = pearsonDF.reset_index(drop=True)


In [101]:
print(pearsonDF['userId'].head())

0    (1,)
1    (4,)
2    (5,)
3    (6,)
4    (7,)
Name: userId, dtype: object


In [102]:
# استخراج مقدار عددی از تاپل
pearsonDF['userId'] = pearsonDF['userId'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

# سپس تبدیل به int
pearsonDF['userId'] = pearsonDF['userId'].astype(int)


In [103]:

# انتخاب کاربران مشابه
topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]


In [104]:

# ادغام امتیازات کاربران مشابه با دیتافریم 'ratings_df'
topUsersRating = pd.merge(topUsers, ratings_df, left_on='userId', right_on='userId', how='inner')


In [105]:

# محاسبه امتیازات وزنی
topUsersRating['weightedRating'] = topUsersRating['similarityIndex'] * topUsersRating['rating']


In [106]:

# گروه‌بندی بر اساس 'movieId' و محاسبه مجموع امتیازات وزنی و شاخص شباهت
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex', 'weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex', 'sum_weightedRating']

In [107]:

# محاسبه امتیاز نهایی توصیه‌ها
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating'] / tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index


In [108]:


# مرتب‌سازی و نمایش 10 فیلم برتر پیشنهادی
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommended_movies = movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

# نمایش فیلم‌های پیشنهادی
print(recommended_movies)


      movieId                                      title  year
27         28                                 Persuasion  1995
76         85                         Angels and Insects  1995
2215     2943                                  Indochine  1992
2279     3022                               General, The  1926
2288     3035                             Mister Roberts  1955
9367   162414                                  Moonlight   NaN
9443   167064                        I Am Not Your Negro  2017
9611   176601                               Black Mirror   NaN
9618   177593  Three Billboards Outside Ebbing, Missouri  2017
9711   187717                  Won't You Be My Neighbor?  2018
